In [ ]:
import time
membership_dict = {}
menu_list = [
    {'name': '바나나', 'price': 4000},
    {'name': '딸기', 'price': 5000},
    {'name': '벌집꿀', 'price': 7000},
    {'name': '그레놀라', 'price': 7000},
    {'name': '초코쉘', 'price': 7000},
    {'name': '요거트 아이스크림', 'price': 3000}
]

def get_valid_number_input(msg, min_no = 1, max_no = 5):
    # 숫자체크
    try:
        number = int(input(msg))
        if number < min_no:
            print(f"{min_no} 이상 숫자를 입력하세요")
            return None
        elif number > max_no:
            print(f"{max_no} 이하 숫자를 입력하세요")
            return None
        else:
            return number
    except ValueError:
        print("숫자를 입력하세요")
        return None

def display_menu():
    print('*' * 40)
    print('어서오세요, ‘요렇게’에 오신 것을 환영합니다:)')
    print("Menu\t\t\t\tPrice")
    print("-" * 30)
    print("요거트 아이스크림(250g)\t3,000")
    print("-" * 30)
    print("\nNo. Topping\t\tPrice")
    print("-" * 30)
    for idx, item in enumerate(menu_list, start=1):
        print(f"{idx:<5} {item['name']:<10}\t{item['price']:,}")
    print("-" * 30)
    print("*" * 40)

def display_cart(cart, total_price):
    print("\n" + "=" * 40)
    print("현재 장바구니")
    for i, item in enumerate(cart, start=1):
        print(f"{i}. {item['name']} x {item['quantity']} = {item['price'] * item['quantity']:,}원")
    print("-" * 40)
    print(f"Total = {total_price:,}원")

def add_menu(cart, total_price):
    current_total = sum(item['quantity'] for item in cart)
    remaining = 16 - current_total
    
    if remaining <= 0:
        print("장바구니에는 최대 15개까지만 담을 수 있습니다.")
        return total_price

    print("\n메뉴 목록:")
    for i, item in enumerate(menu_list, start=1):
        print(f"{i}. {item['name']} : {item['price']:,}원")
    idx = get_valid_number_input("추가할 토핑 번호를 골라주세요 : ", 1, len(menu_list))
    if idx:
        idx -= 1
        qty = get_valid_number_input(f"수량을 선택해주세요 (남은 수량: {remaining}) : ",1,remaining)
        if qty:
            selected_item = menu_list[idx]
            for item in cart:
                if item['name'] == selected_item['name']:
                    item['quantity'] += qty
                    if item['quantity'] > 15:
                        item['quantity'] = 15
                    break
            else:
                cart.append({'name': selected_item['name'], 'quantity': qty, 'price': selected_item['price']})
    
            total_price += selected_item['price'] * qty
    return total_price

def del_menu(cart, total_price):
    display_cart(cart, total_price)
    idx = get_valid_number_input("삭제할 메뉴의 번호를 골라주세요 : ",1,len(cart))
    if idx:
        idx -= 1
        item = cart[idx]
        print(f"'{item['name']}'의 현재 수량: {item['quantity']}")
        del_qty = get_valid_number_input("몇 개를 삭제하시겠습니까? : ", 1, item['quantity'])
        if del_qty:
            item['quantity'] -= del_qty
            total_price -= item['price'] * del_qty
            print(f"'{item['name']}' {del_qty}개가 삭제되었습니다.")
    return total_price

def select_pro():
    print("메뉴를 확정하려면 o")
    print("메뉴를 삭제하려면 d")
    print("메뉴를 추가하려면 a")
    print("메뉴를 취소하려면 c")
    return input("알파벳을 입력해주세요 : ").lower().strip()

def pay_deci():
    return input("주문이 확정되었습니다. 결제를 하시겠습니까? (y/n): ").lower().strip()

def pay_screen(total_price, num_order):
    phone = None
    used_point = 0

    use_point = input("포인트를 사용하시겠습니까? (y/n): ").lower().strip()
    if use_point == 'y':
        phone = input("전화번호를 입력해주세요 (- 없이): ").strip()
        if phone in membership_dict:
            available = membership_dict[phone]
            print(f"현재 {available}포인트가 있습니다.")
            while True:
                try:
                    used_point = int(input("사용할 포인트를 입력해주세요: "))
                    if used_point > available:
                        print("포인트가 부족합니다.")
                    elif used_point > total_price:
                        print("결제 금액보다 많은 포인트는 사용할 수 없습니다.")
                    else:
                        membership_dict[phone] -= used_point
                        total_price -= used_point
                        print(f"{used_point}포인트를 사용하여 {total_price}원을 결제합니다.")
                        break
                except ValueError:
                    print("숫자를 입력해주세요.")
        else:
            print("해당 번호로 적립된 포인트가 없습니다.")

    print(f"{total_price:,}원 결제하겠습니다. 카드를 삽입해주세요.")
    print("결제가 완료되었습니다.")

    # 포인트를 사용하지 않았을 때만 적립
    if used_point == 0:
        mem = input("포인트 적립 하시겠습니까? (y/n): ").lower().strip()
        if mem == 'y':
            if not phone:
                phone = input("전화번호를 입력해주세요 (- 없이): ").strip()
            point = int(total_price * 0.1)
            membership_dict[phone] = membership_dict.get(phone, 0) + point
            print(f"{phone} 번호에 {point}포인트가 적립되어 총 {membership_dict[phone]}포인트가 있습니다.")
    else:
        print("포인트를 사용하셨기 때문에 이번 결제는 적립되지 않습니다.")

    print(f"주문번호는 {num_order}번 입니다.")

def membership_point_save(total_price):
    phone = input("전화번호를 입력해주세요 (- 없이): ").strip()
    points = int(total_price * 0.1)

    if phone in membership_dict:
        membership_dict[phone] += points
        print(f"{phone} 번호에 {points}포인트가 추가되어, 총 {membership_dict[phone]}포인트가 있습니다.")
    else:
        membership_dict[phone] = points
        print(f"{phone} 번호로 {points}포인트가 새로 적립되었습니다.")
        
def use_membership_point(total_price):
    phone = input("전화번호를 입력해주세요 (- 없이): ").strip()
    if phone in membership_dict:
        available = membership_dict[phone]
        print(f"{phone}님의 사용 가능한 포인트: {available}점")

        if available == 0:
            print("사용 가능한 포인트가 없습니다.")
            return total_price  # 그대로 반환

        use = get_valid_number_input(
            f"얼마나 사용하시겠습니까? (최대 {min(available, total_price)}원): ",
            1, min(available, total_price)
        )
        if use:
            membership_dict[phone] -= use
            total_price -= use
            print(f"{use}포인트를 사용했습니다. 남은 결제 금액: {total_price:,}원")
    else:
        print("등록된 멤버십 정보가 없습니다.")
    return total_price
    
    
def end_screen_delay():
    print("초기화면으로 돌아갑니다.")
    for i in range(5, 0, -1):
        print(f"{i} ", end='\r', flush=True)
        time.sleep(1)
    print(" ", end='\r')

def main():
    num_order = 1
    while True:
        display_menu()
        cart = [{'name': menu_list[5]['name'], 'price': menu_list[5]['price'], 'quantity': 1}]
        total_price = 3000
        while True:
            display_cart(cart, total_price)
            choice = select_pro()

            if choice == 'o':
                break
            elif choice == 'd':
                total_price = del_menu(cart, total_price)
            elif choice == 'a':
                total_price = add_menu(cart, total_price)
            elif choice == 'c':
                print("주문이 취소됩니다.")
                end_screen_delay()
                break
            elif choice == '99':
                print("관리자 중지입니다.")
                return
            else:
                print("입력이 올바르지 않습니다.")

        if choice in ['c', '99']:
            continue
        if total_price == 0:
            print("장바구니가 비어있습니다.")
            end_screen_delay()
            continue

        if pay_deci() == 'y':
            pay_screen(total_price, num_order)
            num_order += 1
        elif pay_deci() != 'n' :
            print("올바른 알파벳을 입력하세요.")
        end_screen_delay()

main()

****************************************
어서오세요, ‘요렇게’에 오신 것을 환영합니다:)
Menu				Price
------------------------------
요거트 아이스크림(250g)	3,000
------------------------------

No. Topping		Price
------------------------------
1     바나나       	4,000
2     딸기        	5,000
3     벌집꿀       	7,000
4     그레놀라      	7,000
5     초코쉘       	7,000
6     요거트 아이스크림 	3,000
------------------------------
****************************************

현재 장바구니
1. 요거트 아이스크림 x 1 = 3,000원
----------------------------------------
Total = 3,000원
메뉴를 확정하려면 o
메뉴를 삭제하려면 d
메뉴를 추가하려면 a
메뉴를 취소하려면 c


알파벳을 입력해주세요 :  o
주문이 확정되었습니다. 결제를 하시겠습니까? (y/n):  y
포인트를 사용하시겠습니까? (y/n):  y
전화번호를 입력해주세요 (- 없이):  01022741011


해당 번호로 적립된 포인트가 없습니다.
3,000원 결제하겠습니다. 카드를 삽입해주세요.
결제가 완료되었습니다.


포인트 적립 하시겠습니까? (y/n):  y


01022741011 번호에 300포인트가 적립되어 총 300포인트가 있습니다.
주문번호는 1번 입니다.
초기화면으로 돌아갑니다.
****************************************
어서오세요, ‘요렇게’에 오신 것을 환영합니다:)
Menu				Price
------------------------------
요거트 아이스크림(250g)	3,000
------------------------------

No. Topping		Price
------------------------------
1     바나나       	4,000
2     딸기        	5,000
3     벌집꿀       	7,000
4     그레놀라      	7,000
5     초코쉘       	7,000
6     요거트 아이스크림 	3,000
------------------------------
****************************************

현재 장바구니
1. 요거트 아이스크림 x 1 = 3,000원
----------------------------------------
Total = 3,000원
메뉴를 확정하려면 o
메뉴를 삭제하려면 d
메뉴를 추가하려면 a
메뉴를 취소하려면 c


...


현재 장바구니
1. 요거트 아이스크림 x 1 = 3,000원
2. 초코쉘 x 10 = 70,000원
----------------------------------------
Total = 73,000원
메뉴를 확정하려면 o
메뉴를 삭제하려면 d
메뉴를 추가하려면 a
메뉴를 취소하려면 c


알파벳을 입력해주세요 :  o
주문이 확정되었습니다. 결제를 하시겠습니까? (y/n):  y
포인트를 사용하시겠습니까? (y/n):  y
전화번호를 입력해주세요 (- 없이):  01022741011


현재 300포인트가 있습니다.


사용할 포인트를 입력해주세요:  300


300포인트를 사용하여 72700원을 결제합니다.
72,700원 결제하겠습니다. 카드를 삽입해주세요.
결제가 완료되었습니다.
포인트를 사용하셨기 때문에 이번 결제는 적립되지 않습니다.
주문번호는 2번 입니다.
초기화면으로 돌아갑니다.
****************************************
어서오세요, ‘요렇게’에 오신 것을 환영합니다:)
Menu				Price
------------------------------
요거트 아이스크림(250g)	3,000
------------------------------

No. Topping		Price
------------------------------
1     바나나       	4,000
2     딸기        	5,000
3     벌집꿀       	7,000
4     그레놀라      	7,000
5     초코쉘       	7,000
6     요거트 아이스크림 	3,000
------------------------------
****************************************

현재 장바구니
1. 요거트 아이스크림 x 1 = 3,000원
----------------------------------------
Total = 3,000원
메뉴를 확정하려면 o
메뉴를 삭제하려면 d
메뉴를 추가하려면 a
메뉴를 취소하려면 c


In [1]:
# 유틸리티 함수

# yes or no
def get_yes_or_no_input(msg):
    while True:
        user_input = input('>>>' + msg + " (yes/no): ").strip().lower()
        if user_input in ['yes', 'no']:
            return user_input
        else:
            print("⚠️  'yes' 또는 'no'만 입력할 수 있어요.")


#숫자검증
def get_valid_number_input(msg, min_no=0, max_no=5):
    try:
        number = int(input('>>>' + msg).strip())
        if number < min_no:
            print(f'{min_no} 이상 숫자를 입력하세요.')
            return None
        elif number > max_no:
            print(f'{max_no} 이하 숫자를 입력하세요.')
            return None
        else:
            return number
    except ValueError:
        print('숫자로 입력하세요.')
        return None

In [ ]:
# 데이터 초기화
cart = {'user_menu': '', 'user_size': '', 'qty': ''}
cart_list = []
membership_dict = {}

menu_list = ['초코', '바닐라', '딸기', '우유', '말차', '민트초코']
size_price = {'S': 2000, 'M': 3000, 'L': 4000}
size_list = list(size_price.keys())

# 멤버십 적립 함수
def handle_membership():
    while True:
        member = input("멤버십을 적립하시겠습니까? (yes/no): ").strip().lower() 
        if member not in ['yes', 'no']:
            print("yes 또는 no만 입력해주세요.")
            continue
        if member == 'no':
            print("멤버십 적립을 건너뜁니다.\n")
            return
        else:
            print("멤버십 적립을 선택하셨습니다.")
            while True:
                phone = input("전화 번호를 11자리를 입력하세요: ").strip()
                if len(phone) != 11 or not phone.isdigit():
                    print("전화번호가 올바르지 않습니다. 11자리 숫자만 입력해주세요.")
                    continue
                if phone in membership_dict:
                    print(f"기존 ({phone}) 번호의 멤버십이 있으므로 포인트를 적립했습니다.\n")
                else:
                    membership_dict[phone] = {}
                    print(f"새로운 ({phone}) 번호로 멤버십이 생성되었습니다.\n")
                return

# 장바구니 출력
def print_cart():
    if not cart_list:
        print("\n장바구니가 비어 있습니다.")
        return
    print("\n🛒 현재 장바구니:")
    for i, item in enumerate(cart_list, 1):
        print(f"{i}. {item['user_menu']} ({item['user_size']}) x {item['qty']}개 - {item['total_price']:,}원")
    total = sum(item['total_price'] for item in cart_list)
    print(f"총 합계: {total:,}원")

# 장바구니 보기 및 결제
def function5():
    if not cart_list:
        print("\n장바구니가 비어 있습니다.")
        start()
        return 
    print_cart()
    while True:
        ask_mul = '1. 결제하기  2. 계속 쇼핑하기  0: '
        choice = get_valid_number_input(ask_mul, 0, 2) ## 예외처리 (0~3)
        if choice == 1:
            total = sum(item['total_price'] for item in cart_list)
            print(f"\n총 결제 금액: {total:,}원")

            answer = get_yes_or_no_input("결제하시겠습니까? (yes/no): ") #예외처리 (yes or no)
            if answer not in ['yes', 'no']:
                print("yes 또는 no만 입력해주세요.")
                continue
            elif answer == 'no':
                print("주문이 취소되었습니다.\n")
                return start()

            # 멤버십 적립 처리
            handle_membership()

            print("결제가 완료되었습니다. 감사합니다! 🍦\n")
            cart_list.clear()
            start()
            return

        elif choice == 2:
            return function_1()
        elif choice == 0:
            print("\n이용해주셔서 감사합니다! 좋은 하루 되세요! 👋")
            exit()

# 장바구니 상품 삭제
def function4():
    if not cart_list:
        print("\n장바구니가 비어 있습니다.")
        return
    print_cart()
    try:
        idx = int(input("삭제할 상품 번호를 입력하세요 (취소하려면 0 입력): ")) #예외처리 못함 ㅜㅡㅜ
        if idx == 0:
            print("삭제를 취소했습니다.")
            return
        if 1 <= idx <= len(cart_list):
            removed = cart_list.pop(idx - 1)
            print(f"{removed['user_menu']} ({removed['user_size']}) x {removed['qty']}개가 장바구니에서 삭제되었습니다.")
        else:
            print("⛔ 유효한 번호를 입력하세요.")
    except ValueError:
        print("⛔ 숫자를 입력해주세요.")

# 장바구니에 추가 및 장바구니 기능 선택
def function_3():
    item = {
        'user_menu': cart['user_menu'],
        'user_size': cart['user_size'],
        'qty': cart['qty'],
        'price_per_item': size_price[cart['user_size']],
        'total_price': size_price[cart['user_size']] * cart['qty']
    }
    cart_list.append(item)

    print("\n🍦 장바구니에 추가되었습니다! 🍦")
    print_cart()

    while True:
        cho_mul = "\n1. 계속 쇼핑하기  2. 장바구니 보기 및 결제  3. 장바구니에서 상품 삭제  0. 종료\n선택: "
        choice = get_valid_number_input(cho_mul, 0, 3) ## 예외처리 (0~3)
        if choice == 1:
            function_1()
            return
        elif choice == 2:
            function5()
            return
        elif choice == 3:
            function4()
        elif choice == 0:
            print("\n이용해주셔서 감사합니다! 좋은 하루 되세요! 👋")
            exit()

# 수량 선택
def function_2():
    while True:
        print("\n🍨 사이즈 및 가격\n")
        for i, (size, price) in enumerate(size_price.items()):
            print(f'{i + 1}. 사이즈: {size}, 가격: {price:,}원')
        size_mul = f"\n원하시는 사이즈를 선택해주세요. (1~{len(size_price)}, 0:뒤로가기): "
        size_index = get_valid_number_input(size_mul, 0,3 )## 예외처리 (0~3)
        
        if size_index is None:
            print("⛔ 입력 오류⛔ 사이즈 다시 선택하세요.")

        elif size_index == 0:
            print('\n뒤로 돌아갑니다 ⬅️')
            function_1()
            return
        elif 1 <= size_index <= len(size_price):
            cart['user_size'] = size_list[size_index - 1]
            break
    while True:
        qty_mul = "🤟 원하시는 개수를 적어주세요 (0:뒤로가기): "
        qty = get_valid_number_input(qty_mul, 0, 10) ## 예외처리 (0~10) 성공
        if qty == None:
            continue
        if qty == 0:
            print('\n사이즈 선택으로 돌아갑니다 ⬅️')
            function_2()
            return
        if qty > 0:
            cart['qty'] = qty
            break
    print('-' * 50)
    function_3()
    

# 메뉴 선택
def function_1():
    while True:
        print("\n📋 메뉴판\n")
        for index, name in enumerate(menu_list):
            print(f'{index + 1}. {name}')
        
        menu_mul = f"\n원하시는 메뉴를 선택해주세요. (1~{len(menu_list)}, 0:처음으로): "
        menu_index = get_valid_number_input(menu_mul, 0, len(menu_list))  # 예외처리 

        if menu_index is None:
            continue  # None일 경우 아무 처리 없이 다시 메뉴 선택

        if menu_index == 0:
            print('\n보고싶을거예요 😭😭\n')
            start()
            return

        if 1 <= menu_index <= len(menu_list):
            cart['user_menu'] = menu_list[menu_index - 1]
            print('-' * 50)
            function_2()
            return
# 시작 화면
def start():
    print('=' * 50)
    store_name = 'I Miss You Cram!🍦'
    print(f'        보고싶었어요! {store_name}')
    print('=' * 50)
    function_1()

# main
start()

        보고싶었어요! I Miss You Cram!🍦

📋 메뉴판

1. 초코
2. 바닐라
3. 딸기
4. 우유
5. 말차
6. 민트초코


>>>
원하시는 메뉴를 선택해주세요. (1~6, 0:처음으로):  5


--------------------------------------------------

🍨 사이즈 및 가격

1. 사이즈: S, 가격: 2,000원
2. 사이즈: M, 가격: 3,000원
3. 사이즈: L, 가격: 4,000원


>>>
원하시는 사이즈를 선택해주세요. (1~3, 0:뒤로가기):  2
>>>🤟 원하시는 개수를 적어주세요 (0:뒤로가기):  3


--------------------------------------------------

🍦 장바구니에 추가되었습니다! 🍦

🛒 현재 장바구니:
1. 말차 (M) x 3개 - 9,000원
총 합계: 9,000원


...

--------------------------------------------------

🍨 사이즈 및 가격

1. 사이즈: S, 가격: 2,000원
2. 사이즈: M, 가격: 3,000원
3. 사이즈: L, 가격: 4,000원


>>>
원하시는 사이즈를 선택해주세요. (1~3, 0:뒤로가기):  5


3 이하 숫자를 입력하세요.
⛔ 입력 오류⛔ 사이즈 다시 선택하세요.

🍨 사이즈 및 가격

1. 사이즈: S, 가격: 2,000원
2. 사이즈: M, 가격: 3,000원
3. 사이즈: L, 가격: 4,000원


>>>
원하시는 사이즈를 선택해주세요. (1~3, 0:뒤로가기):  0



뒤로 돌아갑니다 ⬅️

📋 메뉴판

1. 초코
2. 바닐라
3. 딸기
4. 우유
5. 말차
6. 민트초코


In [ ]:
import time
points = {}
# # 메뉴 저장 리스트 -> 메뉴 추가 시 여기서.
menu = [
    {'name':'샤프', 'price': 6000},
    {'name':'볼펜', 'price': 3000},
    {'name':'지우개', 'price': 1000},
    {'name':'화이트', 'price': 3000},
    {'name':'공책', 'price': 3500}
]

def prt_menu(menu):
    print('╔' + '═' * 40 + '╗')
    print(f'빵꾸똥꾸 문구야'.center(34))
    print(f"에 오신 것을 환영합니다!💥".center(34))
    print('╚' + '═' * 40 + '╝') 
    print(f'{"번호"}\t\t{"제품명"}\t\t{"가격"}')
    print('-'*44)

    for i, m in enumerate(menu):
        print(f'{i+1}\t\t{m["name"]}\t\t{m["price"]:,}원')
    print('-'*44)

def prt_alarm():
    print('='*44)
    print(f'{"[알림]❌ 잘못된 입력입니다.":^35}')
    print(f'{"[알림]✅ 다시 입력해주세요.":^35}')
    print('='*44)

def prt_time():
    for i in range(3, 0, -1):
        print('=' * 44)
        print(f'{i}초 뒤에 시작화면으로 돌아갑니다.'.center(34))
        time.sleep(1)
        
def add_ord():
    while True:        
        try:
            ord_num = int(input('주문하실 제품 번호를 선택하세요: ')) -1
            if not (0 <= ord_num < len(menu)):
                prt_alarm()
                continue
            return ord_num
        except ValueError:
            prt_alarm()
            continue
        
def add_qty():
    while True:
        qty = input('\t수량을 입력하세요(취소: x): ')
        if qty.lower() =="x":
            return qty
        try:
            qty = int(qty)
            if qty > 0:
                return qty
            prt_alarm()
        except ValueError:
            prt_alarm()

def add_lst(order_list, ord_num, qty):
    item = menu[ord_num]
    for i in order_list:
        if i['name'] == item['name']:
            i['qty'] += qty
            return order_list 
    order_list.append({
        'name': item['name'],
        'qty': qty,
        'price': item['price']
    })
    return order_list
    
def yesno():
    while True:
        a=input('더 주문하시겠습니까?(y/n):')
        if a.lower() not in ['y','n']:
            prt_alarm()
            continue
        else:
            return a
            
def calc_cost():
    total = 0   
    print('='*44)
    print(f'\t\t{"주문 내역"}')
    print('-'*44)

    for order in order_list:
        item_total = order['qty'] * order['price']
        total += item_total

        print(f'{order["name"]}\t{order["qty"]}개\t-\t{item_total:,}원')

    print('-'*44)
    print(f'총 결제 금액: {total:,}원')
    print('='*44)
    return total

def prt_points():
    while True:
        ab = input('포인트를 적립하시겠습니까? (y/n): ')
        if ab.lower() not in ['y', 'n']:
            prt_alarm()
            continue
        elif ab.lower()=='n':
            return        
        else:
            while True:
                phone = input('전화번호를 입력해 주세요: ')
                if not phone.isdigit() or len(phone) < 11 or phone == "x":
                    prt_alarm()
                    continue                       
                user_id = f'user_{phone[-4:]}'
                point = int(total * 0.05)                 
                if user_id in points :
                    points[user_id] += point
                else :
                    points[user_id] = point
                print('='*44)
                print(f'🎁{user_id}고객님, {point}점 적립 되었습니다!🎁')
                print(f' 고객님의 총 적립포인트는 {points[user_id]:,}점 입니다!')
                print('='*44)
                return
     
######################################################################################################################################################


def draw_pencil_ascii():
    pencil_art = r"""
        .================ooooooo
 ___   ,'    \_________________________________
/   /-/       /                   ////////////  ''--..._
\___\-\       \                   \\\\\\\\\\\\  __..--'
       `--------------------------''''''''''''''
"""
    print(pencil_art)

draw_pencil_ascii()
while True:
    prt_menu(menu)
    # 다중 주문 저장 리스트
    order_list = []

    while True: 
        
        ord_num = add_ord()         
        menut = menu[ord_num]['name']
        
        print('='*44)
        print(f'{"고객님이 선택하신 제품은":^33}')
        print(f'{menut + " 입니다.":^33}')
        print(f'{"몇 개를 드릴까요?":^33}')
        
        
        qty = add_qty()
        if qty in ('x','X'):
            break
             
        add_lst(order_list, ord_num, qty)
        
        answer = yesno()
        if answer == 'n':
            break
        print('-'*44)

    if qty not in ('x','X'):
        
        total = calc_cost()
        
        prt_points()
        
        print(f'{"결제가 완료되었습니다.":^37}')
        print(f'{"잠시만 기다려주세요. 😊":^37}')

        prt_time()


        .================ooooooo
 ___   ,'    \_________________________________
/   /-/       /                   ////////////  ''--..._
\___\-\       \                   \\\\\\\\\\\\  __..--'
       `--------------------------''''''''''''''

╔════════════════════════════════════════╗
             빵꾸똥꾸 문구야             
         에 오신 것을 환영합니다!💥          
╚════════════════════════════════════════╝
번호		제품명		가격
--------------------------------------------
1		샤프		6,000원
2		볼펜		3,000원
3		지우개		1,000원
4		화이트		3,000원
5		공책		3,500원
--------------------------------------------


주문하실 제품 번호를 선택하세요:  5


          고객님이 선택하신 제품은          
             공책 입니다.             
           몇 개를 드릴까요?            


	수량을 입력하세요(취소: x):  50
더 주문하시겠습니까?(y/n): n


		주문 내역
--------------------------------------------
공책	50개	-	175,000원
--------------------------------------------
총 결제 금액: 175,000원


...

		주문 내역
--------------------------------------------
샤프	20개	-	120,000원
--------------------------------------------
총 결제 금액: 120,000원


포인트를 적립하시겠습니까? (y/n):  y
전화번호를 입력해 주세요:  01022741011


🎁user_1011고객님, 6000점 적립 되었습니다!🎁
 고객님의 총 적립포인트는 14,750점 입니다!
            결제가 완료되었습니다.             
            잠시만 기다려주세요. 😊            
       3초 뒤에 시작화면으로 돌아갑니다.        
       2초 뒤에 시작화면으로 돌아갑니다.        
       1초 뒤에 시작화면으로 돌아갑니다.        
╔════════════════════════════════════════╗
             빵꾸똥꾸 문구야             
         에 오신 것을 환영합니다!💥          
╚════════════════════════════════════════╝
번호		제품명		가격
--------------------------------------------
1		샤프		6,000원
2		볼펜		3,000원
3		지우개		1,000원
4		화이트		3,000원
5		공책		3,500원
--------------------------------------------


주문하실 제품 번호를 선택하세요:  99


         [알림]❌ 잘못된 입력입니다.          
         [알림]✅ 다시 입력해주세요.          
